# 03 - 날짜 데이터 처리


## 1.환경준비

### (1) 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### (2) 데이터 준비

In [ ]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")

# 판매액 계산하기
temp = pd.merge(sales, products)
temp['Amt' ] = temp['Qty'] * temp['Price']
temp['Amt'] = (temp['Amt']/1000).round()  # 단위 1000달러

In [ ]:
products.loc[products['Product_ID'] == 12]

In [ ]:
sales.loc[sales['Store_ID']==1].groupby('Product_ID', as_index = False)['Qty'].sum()

In [ ]:
# 집계
data1 = temp.groupby(['Date', 'Category'], as_index = False)['Amt'].sum()
data2 = temp.groupby(['Date'], as_index = False)['Amt'].sum()

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
data11 = data1.pivot('Date', 'Category', 'Amt').reset_index()
data11.head()

In [ ]:
data = pd.merge(data2, data11)
data.head()

## 2.날짜 요소 추출

### (1) 날짜 타입으로 변환
* pd.to_datetime(날짜데이터, format = '입력되는 날짜 형식')
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [ ]:
data.info()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data.info()

* format = ''
    * pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
    * https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
# pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date)
date

In [ ]:
# 만약 dd-mm-yyyy 이 맞다면!
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date, format = '%d-%m-%Y') #입력받은 날짜 데이터 형식이 '%d-%m-%Y'!
date

### (2) 날짜 요소 추출

In [ ]:
# 연도
date.dt.year

In [ ]:
# 월
date.dt.month

In [ ]:
# 일
date.dt.day

In [ ]:
# 요일
date.dt.weekday

In [ ]:
# 요일 이름
date.dt.day_name()

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data의 Date는 이미 날짜 타입으로 변환되어 있습니다.  
* 다음의 항목을 열로 추가하시오.
    * 요일(이름)
    * 주차

## 3.시간에 따른 흐름 추가하기

### (1) shift
* 시계열 데이터에서 시간의 흐름 전후로 정보를 이동시킬 때 사용
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [ ]:
temp = data.loc[:,['Date','Amt']]

In [ ]:
# 전날 매출액 열을 추가합시다.
temp['Amt_lag'] = temp['Amt'].shift() #default = 1

# 전전날 매출액 열을 추가.
temp['Amt_lag2'] = temp['Amt'].shift(2) # 2행 shift

# 다음날 매출액 열을 추가합시다.
temp['Amt_lag_1'] = temp['Amt'].shift(-1)

temp.head()

### (2) rolling + 집계함수
* 시간의 흐름에 따라 일정 기간 동안 평균을 이동하면서 구하기
* .rolling : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
* rolling(n) : 
    * n 기본값은 1
    * min_periods : 최소 데이터수

In [ ]:
# 7일 이동평균 매출액을 구해 봅시다.
temp['Amt_MA7_1'] = temp['Amt'].rolling(7).mean()
temp['Amt_MA7_2'] = temp['Amt'].rolling(7, min_periods = 1).mean()
temp.head(10)

### (3) diff
* 특정 시점 데이터, 이전시점 데이터와의 차이 구하기
* .diff : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html

In [ ]:
# 7일 이동평균 매출액을 구해 봅시다.
temp['Amt_D1'] = temp['Amt'].diff()
temp['Amt_D2'] = temp['Amt'].diff(2)
temp.head(10)

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data의 Grocery가 매출이 가장 높습니다. 이에 대해서 다음의 열을 추가해 봅시다.

* 전날 매출액
* 7일 전(전주 동 요일) 매출액
* 3일 이동평균 매출액
* 전날대비 매출액 증감여부 (증가 1, 감소 -1, 동일 0)
